### 1. 데이터 불러오기 -> df 만들기

In [1]:
import pandas as pd
# from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv

env = load_dotenv()

# 파일 경로 및 데이터 불러오기
existing_data_path = "./file/테스트 2차자료_상시감사팀.xlsx"
df = pd.read_excel(existing_data_path, header=None)
df = df.iloc[:243]
print(df)
# 컬럼 정의
columns_q = ["idx", "Q1", "Q2", "Q3", "Q4", "Q5", "Q6", "DBRN_NM"]
columns_a = ["idx", "A1", "A2", "A3", "A4", "A5", "A6", "DBRN_NM"]

# 첫 번째 row는 질문 DataFrame
question_df = pd.DataFrame(df.iloc[:1].values, columns=columns_q)

# 나머지 row는 응답 DataFrame
answer_df = pd.DataFrame(df.iloc[1:243].values, columns=columns_a)
df


ModuleNotFoundError: No module named 'pandas'

### 2. 직원의견 요약 및 키워드 추출

In [ ]:
import pandas as pd
import json
import os
import time
# from openai import OpenAI
from openai import OpenAI
# from openai._client import OpenAI
# import openai
def prepare_batch(records):
    """
    주어진 레코드 리스트를 OpenAI 배치 처리 API에 적합한 메시지 리스트로 변환.
    """
    messages_batch = []
    for idx, record in enumerate(records):
        custom_id = f"request-{idx + 1}"
        body = {
            "model": "gpt-4o",  # 모델 이름
            "messages": [
                {
                    "role": "system",
                    "content": f"""
당신은 회사 감사실의 직원이자 리스크 관리를 위한 기획자입니다. 직원들에게 6개의 질문을 하고 답변을 받았습니다. 이제 각 질문과 그에 대한 답변을 분석해야 합니다.

# 작업 지시사항:

각 질문과 해당 답변을 인식하십시오.
각 답변을 간결하고 명확하게 요약하고 여러 내용이 있을 경우 모두 출력하십시오. 
요약된 내용에서 3~5개의 주요 키워드를 추출하십시오.
결과를 아래의 JSON 형식으로 출력하십시오.

[출력 형식]
무조건 6개의 딕셔너리 결과가 나와야 합니다.(질문 1~6)

{{
  "질문 1": {{
    "summary": ["요약1","요약2","요약3"...]
    "keywords": ["키워드1", "키워드2", "키워드3"..]
  }},
  "질문 2": {{
    "summary": ["요약1","요약2","요약3"...]
    "keywords": ["키워드1", "키워드2", "키워드3"..]
  }},
  ...
}}

[주의사항]
내용을 보고 불필요한 내용만 존재할 경우 요약, 키워드는 빈칸으로 출력합니다.
입력되는 질문과 답변은 마크다운 형식을 따릅니다.
요약은 각 답변의 핵심 내용을 포괄해야 합니다.
요약에는 점수 내용은 표시하지 않습니다.
키워드는 답변의 주요 주제나 이슈를 반영해야 합니다.
키워드는 요약 내용에서 추출합니다.
JSON 형식이 올바르게 구성되도록 유의하십시오.
출력 시 불필요한 문자나 설명을 포함하지 마십시오.
답변에 xxx 없다는 내용을 표현할 경우 요약 및 키워드에 작성하지 않습니다.
내부통제 관련 부족한 의견을 보기 위한 요청이므로 문제점이라고 생각하는 요약 및 키워드만 추출해주세요.
""",
                },
                {
                    "role": "user",
                    "content": f"""
[질문 & 답변]

## 질문 1
{record['Q1']}

### 답변 1
{record['A1']}

## 질문 2
{record['Q2']}

### 답변 2
{record['A2']}

## 질문 3
{record['Q3']}

### 답변 3
{record['A3']}

## 질문 4
{record['Q4']}

### 답변 4
{record['A4']}

## 질문 5
{record['Q5']}

### 답변 5
{record['A5']}

## 질문 6
{record['Q6']}

### 답변 6
{record['A6']}

""",
                },
            ],
            "max_tokens": 1000,
        }
        message = {
            "custom_id": custom_id,
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": body,
        }
        messages_batch.append(message)
    return messages_batch


def process_batches(client, records, batch_size):
    """
    주어진 레코드를 OpenAI 배치 처리 API에 전달하고 결과를 처리.
    """
    results = []
    for batch_start in range(0, len(records), batch_size):
        batch = records[batch_start : batch_start + batch_size]
        messages_batch = prepare_batch(batch)

        # JSONL 파일로 저장
        with open("batchinput.jsonl", "w", encoding="utf8") as file:
            for item in messages_batch:
                json_string = json.dumps(item, ensure_ascii=False)
                file.write(json_string + "\n")

        # Batch 파일 생성
        batch_input_file = client.files.create(
            file=open("batchinput.jsonl", "rb"), purpose="batch"
        )
        batch_input_file_id = batch_input_file.id

        # Batch 생성
        batch_info = client.batches.create(
            input_file_id=batch_input_file_id,
            endpoint="/v1/chat/completions",
            completion_window="24h",
            metadata={"description": "batch job"},
        )

        # Batch 상태 확인 및 결과 다운로드
        while True:
            batch_status = client.batches.retrieve(batch_info.id).status

            if batch_status == "completed":

                # 기존 데이터 프레임 df 에 llm으로부터 받아온 답변을 추가하기 위한 index
                idx = 0

                print(f"Batch {batch_info.id} processing is completed.")

                # 출력 파일 다운로드
                output_file_id = client.batches.retrieve(batch_info.id).output_file_id
                if output_file_id:
                    result = client.files.content(output_file_id).content
                    result_file_name = f"./batch_results_{batch_start}.jsonl"

                    with open(result_file_name, "wb") as file:
                        file.write(result)

                    print("Output file downloaded:", result_file_name)

                    # 결과 읽기
                    with open(result_file_name, "r", encoding="utf-8") as file:
                        for line in file:
                            json_object = json.loads(line.strip())
                            results.append(json_object)
                break

            elif batch_status in ["failed", "cancelled"]:
                print(f"Batch {batch_info.id} processing failed or was cancelled.")
                break

            else:
                print(
                    f"Batch {batch_info.id} is in progress... Current status: {batch_status}"
                )
                time.sleep(10)

    return results, batch_info


records = [
    {
        "Q1": question_df.loc[0, "Q1"],
        "Q2": question_df.loc[0, "Q2"],
        "Q3": question_df.loc[0, "Q3"],
        "Q4": question_df.loc[0, "Q4"],
        "Q5": question_df.loc[0, "Q5"],
        "Q6": question_df.loc[0, "Q6"],
        "A1": answer_df.loc[i, "A1"],
        "A2": answer_df.loc[i, "A2"],
        "A3": answer_df.loc[i, "A3"],
        "A4": answer_df.loc[i, "A4"],
        "A5": answer_df.loc[i, "A5"],
        "A6": answer_df.loc[i, "A6"],
    }
    for i in range(len(answer_df))
]

# 클라이언트 생성 및 배치 처리 실행
client = OpenAI()
batch_size = 3000
results, batch_info1 = process_batches(client, records, batch_size)

Batch batch_6747d1bb50948190a824305d78b0907d is in progress... Current status: validating
Batch batch_6747d1bb50948190a824305d78b0907d is in progress... Current status: in_progress
Batch batch_6747d1bb50948190a824305d78b0907d is in progress... Current status: in_progress
Batch batch_6747d1bb50948190a824305d78b0907d is in progress... Current status: in_progress
Batch batch_6747d1bb50948190a824305d78b0907d is in progress... Current status: in_progress
Batch batch_6747d1bb50948190a824305d78b0907d is in progress... Current status: in_progress
Batch batch_6747d1bb50948190a824305d78b0907d is in progress... Current status: in_progress
Batch batch_6747d1bb50948190a824305d78b0907d is in progress... Current status: finalizing
Batch batch_6747d1bb50948190a824305d78b0907d is in progress... Current status: finalizing
Batch batch_6747d1bb50948190a824305d78b0907d is in progress... Current status: finalizing
Batch batch_6747d1bb50948190a824305d78b0907d is in progress... Current status: finalizing
Batc

In [ ]:
import re


def save_results_to_dataframe(df, results):
    """
    JSONL 파일 내용을 데이터프레임으로 저장.
    """

    # JSON 파싱

    try:

        # data = json.loads(results)
        results = results

    except json.JSONDecodeError as e:

        print("JSON Decode Error:", e)

        print("Response content:", results)

    i = 1

    for result in results:

        # 결과로 회신받은 jsonl 을 line 별로 처리하여 데이터 추출
        content = result["response"]["body"]["choices"][0]["message"]["content"]
        # json 을 dict 로 불러올 경우 발생할 수 있는 오류 인자 제거
        content = content.strip()

        if not content.startswith("{") or not content.endswith("}"):
            # JSON만 추출
            json_match = re.search(r"\{.*\}", content, re.DOTALL)
            if json_match:
                content = json_match.group(0)
            else:
                print("No JSON object found in response.")
                print("Response content:", content)
                raise ValueError("Response is not a valid JSON format.")
        
        # 쉼표가 누락된 경우 수정
        def fix_missing_commas(json_string):
            # 줄바꿈 후 중괄호가 나오는 경우 수정
            fixed = re.sub(r"(?<![:,\[\{\s])(\s*[\{\[])", r",\1", json_string)
            # 중괄호 전에 누락된 쉼표 수정
            fixed = re.sub(r"([\}\]])\s*([\{\[])",
                           r"\1,\2", fixed)
            
            fixed = re.sub(
                r'(\])\s*("[a-zA-Z가-힣0-9_-]+")',
                r'\1,\2',
                fixed
            )
            fixed = fixed.replace(',{', '{')
            return fixed

        content = fix_missing_commas(content)

        # json 을 읽어와 dict 형테로 저장
        try :
            data = json.loads(content)
        except json.JSONDecodeError as e:
            print(content)
            print(i)
        for k in range(1, 7):

            question_key = f"질문 {k}"

            if question_key in data:

                question_data = data[question_key]

                # Summary 변환

                summary_list = question_data["summary"]

                formatted_summary = "\n".join(
                    [f"{i+1}. {summary}" for i, summary in enumerate(summary_list)]
                )

                df.at[i, f"s{k}"] = formatted_summary

                # Keywords 변환

                keywords_list = question_data["keywords"]

                formatted_keywords = "\n".join(
                    [f"{i+1}. {keyword}" for i, keyword in enumerate(keywords_list)]
                )

                df.at[i, f"k{k}"] = formatted_keywords

            else:

                df.at[i, f"s{k}"] = "N/A"

                df.at[i, f"k{k}"] = "N/A"

        i += 1
    return df

In [ ]:
df = save_results_to_dataframe(df, results)
df_copy = df
df_copy = df_copy[1:]
df_copy.rename(columns={0: "idx"}, inplace=True)

C:\Users\shic\AppData\Local\Temp\ipykernel_19548\1682641977.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy.rename(columns={0: "idx"}, inplace=True)


In [ ]:
# df_keyword = pd.DataFrame(
#     df_copy.values, columns=["idx", "k1", "k2", "k3", "k4", "k5", "k6"]
# )
columns = ["idx", "k1", "k2", "k3", "k4", "k5", "k6"]
df_keyword = df_copy[columns]
# k1~k6 컬럼에 있는 키워드를 정리하는 함수


# 숫자 제거 및 키워드 리스트로 변환하는 함수
def process_keywords(col):
    if pd.notna(col):  # 값이 NaN이 아닌 경우 처리
        return [
            item.split(". ", 1)[1] if ". " in item else item for item in col.split("\n")
        ]
    return []


# 각 km_1 ~ km_6에 저장
for i in range(1, 7):
    col_name = f"k{i}"  # 기존 컬럼 이름
    new_col_name = f"km_{i}"  # 새로운 컬럼 이름
    df_keyword[new_col_name] = df_keyword[col_name].apply(process_keywords)

# 모든 키워드를 합쳐서 km_f에 저장
df_keyword["km_f"] = df_keyword[[f"km_{i}" for i in range(1, 7)]].sum(axis=1)


# 빈 값(",")을 제거하는 함수
def remove_empty_keywords(keywords):
    return [kw.strip() for kw in keywords if kw.strip() not in [",", ""]]


# 컬럼에 함수 적용
df_keyword["km_f"] = df_keyword["km_f"].apply(remove_empty_keywords)
df_keyword.drop_duplicates(subset=["km_f"], inplace=True)

# 결과 출력
print(df_keyword["km_f"])
df_keyword.to_csv("keyword.csv")

1                                                     []
2                               [일률적 처리, 본사와 지점 괴리, 투명성]
3                                  [준법통제, 인력 양성, 전문적 통제]
4                               [사기 저하, 관리감독 부재, 로열티 문제]
5                                          [KPI, 부담, 고민]
                             ...                        
237                                   [부서 수익, 내부통제, 리스크]
238    [승인 프로세스, 개선 필요, 시스템 구축, 전자문서 시스템, 업무 프로세스 매뉴얼...
240    [승인 프로세스, 개선 필요, 시스템 구축, 업무 개선, 전자문서, 본사부서 지원,...
241    [책임자 부재, 승인 프로세스, 개선 필요, 해외주식, 단가입력, 시스템구축, 대주...
242                [라임사태, 사모펀드, 서류 간소화, 부서 수익, 내부통제 리스크]
Name: km_f, Length: 232, dtype: object


C:\Users\shic\AppData\Local\Temp\ipykernel_19548\964512619.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_keyword[new_col_name] = df_keyword[col_name].apply(process_keywords)
C:\Users\shic\AppData\Local\Temp\ipykernel_19548\964512619.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_keyword[new_col_name] = df_keyword[col_name].apply(process_keywords)
C:\Users\shic\AppData\Local\Temp\ipykernel_19548\964512619.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

### 3. 키워드별 직원 맵핑, 지점 맵핑

In [ ]:
import pandas as pd
from konlpy.tag import Okt



df_keyword


,idx,k1,k2,k3,k4,k5,k6,km_1,km_2,km_3,km_4,km_5,km_6,km_f
1,1,,,,,,,[],[],[],[],[],[],[]
2,2,,,,,,1. 일률적 처리\n2. 본사와 지점 괴리\n3. 투명성,[],[],[],[],[],"[일률적 처리, 본사와 지점 괴리, 투명성]","[일률적 처리, 본사와 지점 괴리, 투명성]"
3,3,,,,,,1. 준법통제\n2. 인력 양성\n3. 전문적 통제,[],[],[],[],[],"[준법통제, 인력 양성, 전문적 통제]","[준법통제, 인력 양성, 전문적 통제]"
4,4,,,,,,1. 사기 저하\n2. 관리감독 부재\n3. 로열티 문제,[],[],[],[],[],"[사기 저하, 관리감독 부재, 로열티 문제]","[사기 저하, 관리감독 부재, 로열티 문제]"
5,5,,,,1. KPI\n2. 부담\n3. 고민,,,[],[],[],"[KPI, 부담, 고민]",[],[],"[KPI, 부담, 고민]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,237,,,,,,1. 부서 수익\n2. 내부통제\n3. 리스크,[],[],[],[],[],"[부서 수익, 내부통제, 리스크]","[부서 수익, 내부통제, 리스크]"
238,238,1. 승인 프로세스\n2. 개선 필요,,1. 시스템 구축\n2. 전자문서 시스템\n3. 업무 프로세스 매뉴얼\n4. 담당자...,,,1. 순환보직,"[승인 프로세스, 개선 필요]",[],"[시스템 구축, 전자문서 시스템, 업무 프로세스 매뉴얼, 담당자 최신화]",[],[],[순환보직],"[승인 프로세스, 개선 필요, 시스템 구축, 전자문서 시스템, 업무 프로세스 매뉴얼..."
240,240,1. 승인 프로세스\n2. 개선 필요,,1. 시스템 구축\n2. 업무 개선\n3. 전자문서\n4. 본사부서 지원,,,1. 소통 어려움\n2. 순환보직,"[승인 프로세스, 개선 필요]",[],"[시스템 구축, 업무 개선, 전자문서, 본사부서 지원]",[],[],"[소통 어려움, 순환보직]","[승인 프로세스, 개선 필요, 시스템 구축, 업무 개선, 전자문서, 본사부서 지원,..."
241,241,1. 책임자 부재\n2. 승인 프로세스\n3. 개선 필요,,1. 해외주식\n2. 단가입력\n3. 시스템구축\n4. 대주주 대출\n5. 업무개선...,,,1. 업무팀\n2. 순환보직\n3. 개선 필요,"[책임자 부재, 승인 프로세스, 개선 필요]",[],"[해외주식, 단가입력, 시스템구축, 대주주 대출, 업무개선, 망고매뉴얼, 알프, 내...",[],[],"[업무팀, 순환보직, 개선 필요]","[책임자 부재, 승인 프로세스, 개선 필요, 해외주식, 단가입력, 시스템구축, 대주..."


In [ ]:
import pandas as pd

# 키워드 합치기
import ast

df_keyword = pd.read_csv("./keyword.csv")
# df 에 저장된 리스트 형식 텍스트를 실제 list 로 변환
merged_keywords = []
df_keyword["km_f"] = df_keyword["km_f"].apply(ast.literal_eval)
for i in range(1,7):
    df_keyword[f"km_{i}"] = df_keyword[f"km_{i}"].apply(ast.literal_eval)

for row in df_keyword["km_f"]:

    for i in row:

        merged_keywords.append(i)


merged_keywords

['일률적 처리',
 '본사와 지점 괴리',
 '투명성',
 '준법통제',
 '인력 양성',
 '전문적 통제',
 '사기 저하',
 '관리감독 부재',
 '로열티 문제',
 'KPI',
 '부담',
 '고민',
 '구조적 어려움',
 '부당한 업무지시',
 '부적절한 업무',
 '면담 프로세스',
 '이해 부족',
 '자기 주도 개선',
 '업무 프로세스',
 '상식적 절차',
 '대출',
 '민원',
 '트라우마',
 '시스템',
 '개선',
 '인수인계',
 '교육 필요성',
 '현장점검 문제',
 '원본문서 보관',
 '서류 보완',
 '유선업무 확대',
 '유선 녹취',
 '메신져 기록',
 '리스크 발생 우려',
 '상품 선가입',
 '내부통제 미비',
 '보완',
 '계열사 업무',
 '메신져 처리',
 '절차적 문제',
 '상품 선가입',
 '내부통제 미비',
 '보완',
 '퇴직연금',
 '자동 발송',
 '기능 필요',
 '금융사고',
 '투명성',
 '공유 문화',
 '인감 보관',
 '날인',
 '리스크',
 '상품 선가입',
 '보완 필요',
 '오류',
 '수기 업무',
 '해외주식',
 '단가 입력',
 '대출 자금',
 '예수금대체',
 '녹취 점검',
 '가족 계좌개설',
 '전문투자자 등록',
 '실물 확인',
 '권리 누락',
 '책임 소재',
 '혼선',
 '상품 선가입',
 '내부통제 미비',
 '경각심',
 '게시물',
 '사고 예방',
 '상품 가입 미비',
 '부적절한 절차',
 'PB 골+ 교육',
 '지점 축소',
 '업무 소화 지원',
 '금융상품 가입',
 '시간 소요',
 '법인금액 보완',
 '내부통제',
 '준수',
 '시장 리스크 관리',
 '내부통제',
 '영업 어려움',
 '전화녹취',
 '내방 업무',
 '개선 필요',
 '전문가 영업',
 '잘못된 인사',
 '인력 배치',
 'IT 문제',
 '내부통제 절차',
 '준수 어려움',
 'LP업무',
 '권한',
 '한도',
 '내부통제',
 '내부통

In [ ]:
# import openai
from openai import OpenAI
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
from collections import defaultdict


from dotenv import load_dotenv

load_dotenv()
client = OpenAI()
# client = OpenAI()
# 1. 키워드 데이터
keywords = []
# df_keyword["km_f"]에 저장된 키워드를 모두 합치고 중복 제거
for row in df_keyword["km_f"]:
    for i in row:
        keywords.append(i)
keywords = list(set(keywords))
# 2. OpenAI Embedding API를 사용해 키워드 Embedding 생성
def get_embeddings(texts, batch_size=50):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        response = client.embeddings.create(
            model="text-embedding-ada-002",
            input=batch
        )
        # 'data'에 접근할 때는 response.data로 접근해야 함
        for data in response.data:
            embeddings.append(data.embedding)
    return np.array(embeddings)

print("Generating embeddings for keywords...")
embeddings = get_embeddings(keywords)
embeddings = embeddings

# 3. 최적의 클러스터 수 찾기
def find_optimal_clusters(embeddings, max_clusters=200):
    best_score = -1
    best_n_clusters = 2
    for n_clusters in range(200, max_clusters + 1):
        clustering = AgglomerativeClustering(n_clusters=n_clusters, linkage="average")
        labels = clustering.fit_predict(embeddings)
        score = silhouette_score(embeddings, labels, metric="cosine")
        print(f"클러스터 수: {n_clusters}, 실루엣 스코어: {score:.4f}")
        if score > best_score:
            best_score = score
            best_n_clusters = n_clusters
    return best_n_clusters

print("Finding the optimal number of clusters...")
optimal_clusters = find_optimal_clusters(embeddings, max_clusters=300)

# 4. Agglomerative Clustering 적용
print(f"Optimal number of clusters: {optimal_clusters}")
clustering = AgglomerativeClustering(n_clusters=optimal_clusters, linkage="average")
labels = clustering.fit_predict(embeddings)

# 5. 클러스터 결과 정리
cluster_dict = defaultdict(list)
for label, keyword in zip(labels, keywords):
    cluster_dict[label].append(keyword)

# 6. 대표 키워드 추출
def extract_representative_keywords(cluster_dict, embeddings, labels):
    representative_keywords = {}
    for label, keywords in cluster_dict.items():
        cluster_indices = [i for i, l in enumerate(labels) if l == label]
        cluster_embeddings = embeddings[cluster_indices]
        cluster_center = cluster_embeddings.mean(axis=0)
        distances = np.linalg.norm(cluster_embeddings - cluster_center, axis=1)
        closest_index = cluster_indices[np.argmin(distances)]
        representative_keywords[label] = keywords[np.argmin(distances)]
    return representative_keywords

representative_keywords = extract_representative_keywords(cluster_dict, embeddings, labels)

# 7. 결과 출력
for label, keyword_list in cluster_dict.items():
    print(f"\n클러스터 {label + 1}: 대표 키워드 - {representative_keywords[label]}")
    print("키워드들:")
    for keyword in keyword_list:
        print(f"- {keyword}")


Generating embeddings for keywords...
Finding the optimal number of clusters...
클러스터 수: 200, 실루엣 스코어: 0.1652
클러스터 수: 201, 실루엣 스코어: 0.1652
클러스터 수: 202, 실루엣 스코어: 0.1653
클러스터 수: 203, 실루엣 스코어: 0.1652
클러스터 수: 204, 실루엣 스코어: 0.1646
클러스터 수: 205, 실루엣 스코어: 0.1646
클러스터 수: 206, 실루엣 스코어: 0.1666
클러스터 수: 207, 실루엣 스코어: 0.1672
클러스터 수: 208, 실루엣 스코어: 0.1669
클러스터 수: 209, 실루엣 스코어: 0.1660
클러스터 수: 210, 실루엣 스코어: 0.1663
클러스터 수: 211, 실루엣 스코어: 0.1658
클러스터 수: 212, 실루엣 스코어: 0.1661
클러스터 수: 213, 실루엣 스코어: 0.1670
클러스터 수: 214, 실루엣 스코어: 0.1676
클러스터 수: 215, 실루엣 스코어: 0.1679
클러스터 수: 216, 실루엣 스코어: 0.1699
클러스터 수: 217, 실루엣 스코어: 0.1699
클러스터 수: 218, 실루엣 스코어: 0.1707
클러스터 수: 219, 실루엣 스코어: 0.1729
클러스터 수: 220, 실루엣 스코어: 0.1726
클러스터 수: 221, 실루엣 스코어: 0.1735
클러스터 수: 222, 실루엣 스코어: 0.1743
클러스터 수: 223, 실루엣 스코어: 0.1743
클러스터 수: 224, 실루엣 스코어: 0.1741
클러스터 수: 225, 실루엣 스코어: 0.1744
클러스터 수: 226, 실루엣 스코어: 0.1744
클러스터 수: 227, 실루엣 스코어: 0.1746
클러스터 수: 228, 실루엣 스코어: 0.1752
클러스터 수: 229, 실루엣 스코어: 0.1749
클러스터 수: 230, 실루엣 스코어: 0.1746
클러스터 수: 231, 실루엣 스코어:

In [ ]:
# representative_keywords
cluster_dict



# 키를 찾는 함수
def find_key_by_value_list(value_list, dictionary):
    key_list = []
    for value in value_list:
        for key, values in dictionary.items():
            if value in values:
                key_list.append(key)
    return key_list


key_list = []
key_list_1 = []
key_list_2 = []
key_list_3 = []
key_list_4 = []
key_list_5 = []
key_list_6 = []

# 딕셔너리로 변수 초기화
key_lists = {f"key_list_{i}": [] for i in range(1, 7)}

for i in range(len(df_keyword)):
    # print(df_keyword.loc[i, f"km_1"])

    key_list0 = find_key_by_value_list(df_keyword.loc[i, f"km_f"], cluster_dict)
    key_list1 = find_key_by_value_list(df_keyword.loc[i, f"km_1"], cluster_dict)
    key_list2 = find_key_by_value_list(df_keyword.loc[i, f"km_2"], cluster_dict)
    key_list3 = find_key_by_value_list(df_keyword.loc[i, f"km_3"], cluster_dict)
    key_list4 = find_key_by_value_list(df_keyword.loc[i, f"km_4"], cluster_dict)
    key_list5 = find_key_by_value_list(df_keyword.loc[i, f"km_5"], cluster_dict)
    key_list6 = find_key_by_value_list(df_keyword.loc[i, f"km_6"], cluster_dict)

    key_list.append(list(set(key_list0)))
    key_list_1.append(list(set(key_list1)))
    key_list_2.append(list(set(key_list2)))
    key_list_3.append(list(set(key_list3)))
    key_list_4.append(list(set(key_list4)))
    key_list_5.append(list(set(key_list5)))
    key_list_6.append(list(set(key_list6)))
    # print(key_list1)
   


In [ ]:
df_keyword["km_f_gp"] = key_list
df_keyword["km_1_gp"] = key_list_1
df_keyword["km_2_gp"] = key_list_2
df_keyword["km_3_gp"] = key_list_3
df_keyword["km_4_gp"] = key_list_4
df_keyword["km_5_gp"] = key_list_5
df_keyword["km_6_gp"] = key_list_6

df_keyword.to_csv("result_final.csv")


In [ ]:
df_final = pd.read_csv("result_final.csv")

# df 에 저장된 리스트 형식 텍스트를 실제 list 로 변환
df_final["km_f"] = df_final["km_f_gp"].apply(ast.literal_eval)
for i in range(1,7):
    df_final[f"km_{i}_gp"] = df_final[f"km_{i}"].apply(ast.literal_eval)




In [ ]:
df_final["km_f"] = df_final["km_f_gp"].apply(ast.literal_eval)
df_final.loc[0, "km_f"]

[]

In [ ]:
# 7. 결과 출력
for label, keyword_list in cluster_dict.items():
    print(f"\n클러스터 {label + 1}: 대표 키워드 - {representative_keywords[label]}")
    print("키워드들:")
    for keyword in keyword_list:
        print(f"- {keyword}")

columns = ['label','keyword','keywords','cnt','idx_list']
df_result = pd.DataFrame(columns=columns)
# label 별 대표키워드, idx 맵핑 처리
for label, keyword_list in cluster_dict.items():
    cnt = 0
    idx_list = []
    for i in range(len(df_final)) :
        if label in ast.literal_eval( df_final.loc[i,"km_f_gp"]):
            idx_list.append(df_final.loc[i, "idx"])
            cnt += 1

    # 각 행 데이터 생성
    row = {
        'label': label,
        'keyword': representative_keywords[label],  # 대표 키워드
        'keywords': keyword_list,  # 키워드 목록
        'cnt': cnt,  # 키워드 개수
        'idx_list': idx_list  # 키워드의 임의 인덱스
    }
    # DataFrame에 추가
    df_result = pd.concat([df_result, pd.DataFrame([row])], ignore_index=True)




클러스터 82: 대표 키워드 - 업무개선
키워드들:
- 업무능력 향상
- 보완 문제
- 업무 시스템 개선
- 개선 실패
- 업무절차 개선
- 업무개선
- 자기 주도 개선
- 업무 개선
- 업무개선 피드백
- 전달 방법 개선
- 문제 개선
- 부서 관리 개선
- 보완
- 개선

클러스터 72: 대표 키워드 - 대직자 부족
키워드들:
- 정보 부족
- 관심 부족
- 책임 부족
- 견제 부족
- 전문성 부족
- 업무공지 부족
- 현장 이해 부족
- 협업 부족
- 신규고객 부족
- 대안 부족
- 대직자 부족
- 동기부여 부족
- 담당자 부족
- 자원 부족
- 실무교육 부족
- 전문가 부족
- 담보부족
- 경험 부족
- 본사부서 전문성 부족
- 조건 설명 부족
- 이해 부족

클러스터 60: 대표 키워드 - 장기 미비
키워드들:
- 서류 미비
- 장기 미비
- 장기미비
- 상품 가입 미비

클러스터 120: 대표 키워드 - 개선 필요
키워드들:
- 기능 필요
- 전산개발 필요
- 정립 필요
- 개선 필요
- 본사부서 정리 필요
- 필요
- 수시 감사 필요
- 보완 필요
- 문의처 필요
- 기준 필요
- 발전 필요
- 업데이트 필요
- 변경 필요
- 확충 필요
- 경영감사 필요
- 화면 업데이트 필요

클러스터 178: 대표 키워드 - 경쟁력
키워드들:
- 상품 경쟁력
- 금리경쟁력
- 분석 데이터 경쟁력 부족
- 상품 경쟁력 부족
- 경쟁력 문제
- 금리 경쟁력
- 경쟁력 부족
- 금융상품 경쟁력
- 경쟁사 비교
- 경쟁사
- 경쟁력

클러스터 92: 대표 키워드 - 조직 개편
키워드들:
- 조직 문제
- 수직적 조직문화
- 조직 슬림화
- 조직 정비성 부족
- 조직 개편
- 조직 성과
- 조직개편
- 조직문화 개선
- 교육 전담 조직
- 본사조직
- 조직 개편 필요
- 조직 구조 개선
- 조직 시스템
- 전담조직
- 전담조직 필요
- 조직문화
- 조직도 개편
- 조직변경 문제

클러스터 206: 대표 키워드 - 업무 어려움
키워드들:
- 비교 어려움
- 통제 어려움

In [ ]:
file_path = './file/설문 정리.xlsx'
with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name='idx_요약-키워드', index=False)
    df_keyword.to_excel(writer, sheet_name='idx_키워드', index=False)
    df_result.to_excel(writer, sheet_name='키워드 현황', index=False)

# df_result.to_excel("./키워드 현황.xlsx")

PermissionError: [Errno 13] Permission denied: './file/설문 정리.xlsx'